In [ ]:
import pathlib
import sys

In [ ]:
import numpy as np
import joblib
import pandas as pd
from IPython.display import display
from tqdm import tqdm

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "lgbm_reg.joblib"
prediction_path = models_dir_path / "prediction.parquet"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *
from package.feature_extraction import *

In [ ]:
interim = pd.read_parquet(interim_path)

In [ ]:
model = joblib.load(model_path)

In [ ]:
is_test = interim[target].isnull()
dates = interim.loc[is_test, "date"].unique()

In [ ]:
# This process will need to be fixed later
interim.loc[is_test, target] = 0.0

In [ ]:
%%time
for date in tqdm(dates[:test_days]):
    is_date = (interim["date"] >= date - np.timedelta64(max_lags, "D")) & (
        interim["date"] <= date
    )

    intermediate = interim[is_date].copy()

    # See below:
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/139461
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141515
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141545
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141743
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141972
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151339
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151614
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151927
    create_shift_features(intermediate, level_id_cols[11], [target], periods_online)
    create_rolling_features(
        intermediate, level_id_cols[11:], shift_features_online, windows
    )

    is_date = intermediate["date"] == date
    X = intermediate.loc[is_date, features]

    display(X.isnull().sum())

    y_pred = model.predict(X)
    y_pred = y_pred.astype("float32")
    y_pred /= intermediate.loc[is_date, "sell_price"]

    # See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/137217
    y_pred = np.maximum(0.0, y_pred)

    # See below:
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141452
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/142770
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143295
    y_pred *= 1.03

    is_date = interim["date"] == date
    interim.loc[is_date, target] = y_pred

In [ ]:
interim = interim[is_test]

In [ ]:
interim.groupby("date")[target].sum().plot(figsize=(16, 9), grid=True, marker="o")

In [ ]:
interim.groupby(["date", "state_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
interim.groupby(["date", "store_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
interim.groupby(["date", "cat_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
interim.groupby(["date", "dept_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
interim.to_parquet(prediction_path)